In [28]:
from skimage import data, img_as_float
from skimage import exposure

import plotly
from plotly.graph_objs import *

import cv2

import math, os, gc, random
import numpy as np
import nibabel as nib
import os.path

## Tony's get_brain_figure stuff
#from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
#from plotly import tools
#plotly.offline.init_notebook_mode()
#
import networkx as nx
import pandas as pd
import re

In [29]:
os.getcwd()

'/Tony'

In [31]:
"""Applies local equilization to the img's histogram and outputs a .nii file"""
print('Generating Histogram...')
path = os.getcwd()

im = nib.load(path + "/" + "Fear199.nii")

im = im.get_data()
img = im[:,:,:]

shape = im.shape
#affine = im.get_affine()

x_value = shape[0]
y_value = shape[1]
z_value = shape[2]

#####################################################

imgflat = img.reshape(-1)

#img_grey = np.array(imgflat * 255, dtype = np.uint8)

#img_eq = exposure.equalize_hist(img_grey)

#new_img = img_eq.reshape(x_value, y_value, z_value)
#globaleq = nib.Nifti1Image(new_img, np.eye(4))

#nb.save(globaleq, '/home/albert/Thumbo/AutAglobaleq.nii')

######################################################

#clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

img_grey = np.array(imgflat * 255, dtype = np.uint8)
#threshed = cv2.adaptiveThreshold(img_grey, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 0)

cl1 = clahe.apply(img_grey)

#cv2.imwrite('clahe_2.jpg',cl1)
#cv2.startWindowThread()
#cv2.namedWindow("adaptive")
#cv2.imshow("adaptive", cl1)
#cv2.imshow("adaptive", threshed)
#plt.imshow(threshed)

localimgflat = cl1 #cl1.reshape(-1)

newer_img = localimgflat.reshape(x_value, y_value, z_value)
localeq = nib.Nifti1Image(newer_img, np.eye(4))
nib.save(localeq,  "Fear199localeq.nii")

Generating Histogram...


In [33]:
print "done"

done


In [38]:
print newer_img.shape
print type(newer_img)

(540, 717, 1358)
<type 'numpy.ndarray'>


In [36]:
from ndreg import *
import matplotlib
import ndio.remote.neurodata as neurodata
import nibabel as nb

In [47]:
ITKdimensionsIMG = np.rollaxis(newer_img, 2)
print ITKdimensionsIMG.shape

(1358, 540, 717)


In [49]:
## if we have (i, j, k), we want (k, j, i)
new_im = newer_img.swapaxes(0,2) # just swap i and k
print new_im.shape

(1358, 717, 540)


In [51]:
import scipy.misc

In [53]:
plane = 0;
for plane in (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 100, 101, 102, 103, 104):
    output = np.asarray(new_im[plane])
    ## Save as TIFF for Ilastik
    scipy.misc.toimage(output).save('HISTEQoutfile' + "Fear199" + 'ITK' + str(plane) + '.tiff')